In [1]:
import numpy as np
import random
import math
import time

In [2]:
def generate_logit_Y(N,beta,p):
    prob_org = np.random.uniform(0,1,[N,1])
    X = np.random.normal(0,1,[N,p])
    prob_Y = np.exp(np.dot(X,beta))/(1+np.exp(np.dot(X,beta)))
    Y = 1*(prob_org<prob_Y)
    return [X,Y]

In [3]:
def logit_MLE(p,X,Y,N):
    betahat = np.zeros([p,1])
    dist = 1.0
    while(dist>1.0E-6):
        beta0 = betahat
        prob = np.exp(np.dot(X,beta0))/(1+np.exp(np.dot(X,beta0)))
        grad = np.dot(np.transpose(X),(Y-prob))
        diag_prob = prob*(prob-1)
        hess = np.dot(np.transpose(X*diag_prob),X)
        betahat = beta0 - np.dot(np.linalg.inv(hess),grad)
        var = -np.linalg.inv(hess)
        sebeta = np.sqrt(var.diagonal())
        sebeta = np.array([sebeta]).T
        dist = np.abs(np.mean(beta0-betahat))
    return [betahat,sebeta]

In [4]:
def logit_MLE_sample(p,X,Y,N,n):
    betahat = np.zeros([p,1])
    sampleID = np.random.choice(N,n,replace = False)
    Y = Y[sampleID]
    X = X[sampleID,:]
    dist = 1.0
    while(dist>1.0E-6):
        beta0 = betahat
        prob = np.exp(np.dot(X,beta0))/(1+np.exp(np.dot(X,beta0)))
        grad = np.dot(np.transpose(X),(Y-prob))
        diag_prob = prob*(prob-1)
        A = np.zeros([n,n])
        np.fill_diagonal(A,diag_prob)
        hess = np.dot(np.dot(np.transpose(X),A),X)
        betahat = beta0 - np.dot(np.linalg.inv(hess),grad)
        var = -np.linalg.inv(hess)
        sebeta = np.sqrt(var.diagonal())
        sebeta = np.array([sebeta]).T
        dist = np.abs(np.mean(beta0-betahat))
    return [betahat,sebeta]

In [5]:
def bootstrap(N,p,X,Y,boot_n):
    beta_boot_inter = np.zeros([boot_n,p])
    for i in range(boot_n):
        IDindex = np.random.choice(N,N)
        X_boot = np.zeros([N,p])
        Y_boot = np.zeros([N,1])
        for k in range(N):
            X_boot[k,:] = X[IDindex[k],:]
            Y_boot[k] = Y[IDindex[k]]
        [beta_boot,se_boot] = logit_MLE(p,X_boot,Y_boot,N)
        beta_boot_inter[i,:] = np.transpose(beta_boot)
    beta_boot = np.std(beta_boot_inter,0) ## bootstrap的SE
    return beta_boot

In [6]:
def generate_deltaset(N,p,sigma2):
    mean = [0,0,0]
    cov = np.zeros([p,p])
    np.fill_diagonal(cov,sigma2)
    delta=np.random.multivariate_normal(mean,cov,N).T  ##维度是p*N，单独拿出来一列是个列表，必须强制转成矩阵才能方便后续操作
    
    std = np.sqrt(sigma2)
    delta_k = np.zeros([p,p])
    for i in range(p):
        delta_k[i,i] = np.random.normal(0,std)
    return [delta,delta_k]

In [7]:
## 该函数输出Sigma_1_hat, Sigma_2_hat以及由二者组成的渐近方差
## 兰总后来做了稍微调整的算法
def estimate_Sigma(p,N,X,Y,delta,delta_k,sigma2,betahat):
    loss = np.zeros([N,1])   
    loss_delta = np.zeros([N,1])
    Sigma_1_tmp_part1 = np.zeros([N,p,p])
    Sigma_2_tmp_part1 = np.zeros([N,p,p])
    for i in range(N):
        Xbeta = np.dot(X[i,:],betahat)  ## X * beta
        delta_tmp = np.array([delta[:,i]]).T  ## p*1
        Xbeta_delta = np.dot(X[i,:],(betahat+delta_tmp))  ## X *(beta+delta)
        loss[i] = -Y[i]*Xbeta + np.log(1+np.exp(Xbeta))  ## 逻辑回归的损失函数是似然函数的负数
        loss_delta[i] = -Y[i]*Xbeta_delta + np.log(1+np.exp(Xbeta_delta))
        dd = np.dot(delta_tmp,delta_tmp.T) ## delta * t(delta)  p*p    
        Sigma_1_tmp_part1[i,:,:] = dd*((loss_delta[i] - loss[i])**2)  ## p*p
        Sigma_2_tmp_part1[i,:,:] = dd*(loss_delta[i] - loss[i])
    Sigma_1_hat_part1 = np.sum(Sigma_1_tmp_part1,0)/2/N/sigma2/sigma2
    Sigma_2_hat_part1 = np.sum(Sigma_2_tmp_part1,0)/N/sigma2/sigma2
    
    Sigma_1_hat = Sigma_1_hat_part1 - np.eye(p)*np.trace(Sigma_1_hat_part1)/(p+2)
    Sigma_2_hat = Sigma_2_hat_part1 - np.eye(p)*np.trace(Sigma_2_hat_part1)/(p+2)
    
    cov = np.dot(np.dot(np.linalg.inv(Sigma_2_hat),Sigma_1_hat),np.linalg.inv(Sigma_2_hat))/N ## 三明治形式方差
    #cov = np.linalg.inv(Sigma_1_hat)/N     ## 只利用Sigma_1_hat求解方差
    #cov = np.linalg.inv(Sigma_2_hat)/N     ## 只利用Sigma_2_hat求解方差
    return[Sigma_1_hat,Sigma_2_hat,cov]

In [8]:
######### 小样本情况，RMSE，MLE的SE，Bootstrap的SE，Mestimator的SE
N = [10000,20000,50000]
beta = np.array([[0.5],[0.5],[0.5]])
p = np.size(beta)
nsimu = 100
boot_n = 200
f = open("SimuResult_Case1_SmallSample_1.txt", 'w')
f.write('N &');f.write('MLE_rmse &');f.write('MLE_se &');f.write('Boot_se &');f.write('M_se &');f.write('MLE_time &');f.write('Boot_time &');f.write('M_time \\\\ \n');

## 不同方法的估计量
beta_result = np.zeros([len(N),p])
beta_boot_result = np.zeros([len(N),p])
MLE_se_result = np.zeros([len(N),p])
M_se_result = np.zeros([len(N),p])

## 不同方法的时间
boot_time = np.zeros([len(N),1])
mle_time = np.zeros([len(N),1])
m_time = np.zeros([len(N),1])

for i in range(len(N)):
    beta_tmp = np.zeros([nsimu,p])
    beta_boot_tmp = np.zeros([nsimu,p])
    MLE_se_tmp = np.zeros([nsimu,p])
    cov_tmp = np.zeros([nsimu,p])
    
    boot_time_tmp = np.zeros([nsimu,1])
    mle_time_tmp = np.zeros([nsimu,1])
    m_time_tmp = np.zeros([nsimu,1])
    
    sigma2 = N[i]**(-0.25)
    np.random.seed(2020)
    for j in range(nsimu):
        print(N[i],j)
        [X,Y] = generate_logit_Y(N[i],beta,p)
        [delta,delta_k] = generate_deltaset(N[i],p,sigma2)
        start1 = time.time();[betahat,sebeta] = logit_MLE(p,X,Y,N[i]);end1 = time.time()  ## 逻辑回归MLE估计
        start2 = time.time();[Sigma_1_hat,Sigma_2_hat,cov] = estimate_Sigma(p,N[i],X,Y,delta,delta_k,sigma2,betahat);end2 = time.time() ## 我们的方法
        start3 = time.time(); beta_boot = bootstrap(N[i],p,X,Y,boot_n);end3 = time.time()
        beta_tmp[j,:] = np.transpose(betahat)
        cov_tmp[j,:] = np.array(cov.diagonal())
        MLE_se_tmp[j,:] = np.transpose(sebeta)
        beta_boot_tmp[j,:] = np.transpose(beta_boot)

        mle_time_tmp[j] = end1 - start1
        m_time_tmp[j] = end2 - start2
        boot_time_tmp[j] = end3 - start3
    M_se_tmp = np.sqrt(cov_tmp)
    bias_beta = beta_tmp - np.transpose(beta)
    beta_result[i,:] = np.sqrt(np.sum(np.multiply(bias_beta,bias_beta),0)/nsimu)  ## 真实的差距
    beta_boot_result[i,:] = np.mean(beta_boot_tmp,0) ## bootstrap的SE
    MLE_se_result[i,:] = np.mean(MLE_se_tmp,0)    ## MLE的SE
    M_se_result[i,:] = np.mean(M_se_tmp,0)        ## 我们的方法的SE
    
    mle_time[i] = np.mean(mle_time_tmp)
    boot_time[i] = np.mean(boot_time_tmp)
    m_time[i] = np.mean(m_time_tmp)
    
    MLE_RMSE = np.round(beta_result[i,:],decimals = 4)
    BOOT_SE = np.round(beta_boot_result[i,:],decimals = 4)
    MLE_SE = np.round(MLE_se_result[i,:],decimals = 4)
    M_SE = np.round(M_se_result[i,:],decimals = 4)
    
    MLE_time = np.round(mle_time[i],decimals = 3)
    BOOT_time = np.round(boot_time[i],decimals = 3)
    M_time = np.round(m_time[i],decimals = 3)
    f.write(str(N[i])+'&');f.write(f'{MLE_RMSE} &');f.write(f'{MLE_SE} &');f.write(f'{BOOT_SE} &');
    f.write(f'{M_SE} &');f.write(f'{MLE_time} &');f.write(f'{BOOT_time} &');f.write(f'{M_time} \\\\ \n');
f.close()
        

10000 0
10000 1
10000 2
10000 3
10000 4
10000 5
10000 6
10000 7
10000 8
10000 9
10000 10
10000 11
10000 12
10000 13
10000 14
10000 15
10000 16
10000 17
10000 18
10000 19
10000 20
10000 21
10000 22
10000 23
10000 24
10000 25
10000 26
10000 27
10000 28
10000 29
10000 30
10000 31
10000 32
10000 33
10000 34
10000 35
10000 36
10000 37
10000 38
10000 39
10000 40
10000 41
10000 42
10000 43
10000 44
10000 45
10000 46
10000 47
10000 48
10000 49
10000 50
10000 51
10000 52
10000 53
10000 54
10000 55
10000 56
10000 57
10000 58
10000 59
10000 60
10000 61
10000 62
10000 63
10000 64
10000 65
10000 66
10000 67
10000 68
10000 69
10000 70
10000 71
10000 72
10000 73
10000 74
10000 75
10000 76
10000 77
10000 78
10000 79
10000 80
10000 81
10000 82
10000 83
10000 84
10000 85
10000 86
10000 87
10000 88
10000 89
10000 90
10000 91
10000 92
10000 93
10000 94
10000 95
10000 96
10000 97
10000 98
10000 99
20000 0
20000 1
20000 2
20000 3
20000 4
20000 5
20000 6
20000 7
20000 8
20000 9
20000 10
20000 11
20000 12
200

In [9]:
#### 大样本情况
N = [50000,100000,200000]
prop = 0.1
beta = np.array([[0.5],[0.5],[0.5]])
p = np.size(beta)
nsimu = 100
boot_n = 200
f = open("SimuResult_Case1_BigSample_1.txt", 'w')
f.write('N &');f.write('MLE_se &');f.write('Boot_se &');f.write('M_se &');f.write('MLE_time &');f.write('Boot_time &');f.write('M_time \\\\ \n');

## 不同方法的估计量
#beta_result = np.zeros([len(N),p])
beta_boot_result = np.zeros([len(N),p])
MLE_se_result = np.zeros([len(N),p])
M_se_result = np.zeros([len(N),p])

## 不同方法的时间
boot_time = np.zeros([len(N),1])
mle_time = np.zeros([len(N),1])
m_time = np.zeros([len(N),1])

for i in range(len(N)):
    n = int(N[i]*prop)
    #beta_tmp = np.zeros([nsimu,p])
    beta_boot_tmp = np.zeros([nsimu,p])
    MLE_se_tmp = np.zeros([nsimu,p])
    cov_tmp = np.zeros([nsimu,p])
    
    boot_time_tmp = np.zeros([nsimu,1])
    mle_time_tmp = np.zeros([nsimu,1])
    m_time_tmp = np.zeros([nsimu,1])
    
    sigma2 = N[i]**(-0.25)
    np.random.seed(2020)
    for j in range(nsimu):
        print(N[i],j)
        [X,Y] = generate_logit_Y(N[i],beta,p)
        [delta,delta_k] = generate_deltaset(N[i],p,sigma2)
        [betahat,sebeta] = logit_MLE_sample(p,X,Y,N[i],n)  ## 逻辑回归sampleMLE估计
        start1 = time.time();[betahat1,sebeta1] = logit_MLE(p,X,Y,N[i]);end1 = time.time()  ## 逻辑回归MLE估计
        start2 = time.time();[Sigma_1_hat,Sigma_2_hat,cov] = estimate_Sigma(p,N[i],X,Y,delta,delta_k,sigma2,betahat);end2 = time.time() ## 我们的方法
        start3 = time.time(); beta_boot = bootstrap(N[i],p,X,Y,boot_n);end3 = time.time()
        cov_tmp[j,:] = np.array(cov.diagonal())
        MLE_se_tmp[j,:] = np.transpose(sebeta1)
        beta_boot_tmp[j,:] = np.transpose(beta_boot)

        mle_time_tmp[j] = end1 - start1
        m_time_tmp[j] = end2 - start2
        boot_time_tmp[j] = end3 - start3
    M_se_tmp = np.sqrt(cov_tmp)
    beta_boot_result[i,:] = np.mean(beta_boot_tmp,0) ## bootstrap的SE
    MLE_se_result[i,:] = np.mean(MLE_se_tmp,0)    ## MLE的SE
    M_se_result[i,:] = np.mean(M_se_tmp,0)        ## 我们的方法的SE
    
    mle_time[i] = np.mean(mle_time_tmp)
    boot_time[i] = np.mean(boot_time_tmp)
    m_time[i] = np.mean(m_time_tmp)

    BOOT_SE = np.round(beta_boot_result[i,:],decimals = 4)
    MLE_SE = np.round(MLE_se_result[i,:],decimals = 4)
    M_SE = np.round(M_se_result[i,:],decimals = 4)
    
    MLE_time = np.round(mle_time[i],decimals = 3)
    BOOT_time = np.round(boot_time[i],decimals = 3)
    M_time = np.round(m_time[i],decimals = 3)
    f.write(str(N[i])+'&');f.write(f'{MLE_SE} &');f.write(f'{BOOT_SE} &');
    f.write(f'{M_SE} &');f.write(f'{MLE_time} &');f.write(f'{BOOT_time} &');f.write(f'{M_time} \\\\ \n');
f.close()
        

50000 0
50000 1
50000 2
50000 3
50000 4
50000 5
50000 6
50000 7
50000 8
50000 9
50000 10
50000 11
50000 12
50000 13
50000 14
50000 15
50000 16
50000 17
50000 18
50000 19
50000 20
50000 21
50000 22
50000 23
50000 24
50000 25
50000 26
50000 27
50000 28
50000 29
50000 30
50000 31
50000 32
50000 33
50000 34
50000 35
50000 36
50000 37
50000 38
50000 39
50000 40
50000 41
50000 42
50000 43
50000 44
50000 45
50000 46
50000 47
50000 48
50000 49
50000 50
50000 51
50000 52
50000 53
50000 54
50000 55
50000 56
50000 57
50000 58
50000 59
50000 60
50000 61
50000 62
50000 63
50000 64
50000 65
50000 66
50000 67
50000 68
50000 69
50000 70
50000 71
50000 72
50000 73
50000 74
50000 75
50000 76
50000 77
50000 78
50000 79
50000 80
50000 81
50000 82
50000 83
50000 84
50000 85
50000 86
50000 87
50000 88
50000 89
50000 90
50000 91
50000 92
50000 93
50000 94
50000 95
50000 96
50000 97
50000 98
50000 99
100000 0
100000 1
100000 2
100000 3
100000 4
100000 5
100000 6
100000 7
100000 8
100000 9
100000 10
100000 11
